<a href="https://colab.research.google.com/github/kunal266/Algo_Practice/blob/main/Spacy_tut/rasa_nlu_spacy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nest_asyncio==1.3.3
!pip install rasa_nlu[spacy]  #restart runtime after installation and rerun the cell.
!pip install rasa_core
!pip install -U ipython #restart runtime after installation and rerun the cell.

In [ ]:
import rasa_core
import rasa_nlu
import spacy
import sys
python = sys.executable

In [3]:
!python -m spacy download en_core_web_md


     |████████████████████████████████| 120.8 MB 12.0 MB/s 

    Linking successful
    /usr/local/lib/python3.7/dist-packages/en_core_web_md -->
    /usr/local/lib/python3.7/dist-packages/spacy/data/en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



In [4]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- how far
- hey there
- whats up
- hey dude
- goodmorning
- goodevening
- good afternoon
## intent:goodbye
- good by
- later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later
## intent:thanks
- thanks
- thank you
- appreciated
- gracias
## intent:directions
- how do i get to
- i need to get to
- is there a bus to
- how i go take reach
- I need directions to
- how do i get a cab
- where can i get a taxi
- where can i get a bus
-i need to get somewhere
-I am going somewhere
## intent:choosing_item
- 1
- 2
- 3
- 4
- 5
- 6
- 7
## intent:accomodation_enquiry
- Where is the nearest hotel ?
- is there a hotel near by ?
- shey hotel they this area ?
- Where i fit crash for here ?
- i want to get to a hotel
- i need to get to a hotel
- where can i spend the night ?
## intent: restaurant_enquiry
- i need to get food
- i am hungry
- where is the restaurant ?
- is there a restaurant ?
- where can i get water ?
- where i fit buy food for here ?
- is there a food court nearby ?
- where them they sell food ?
- I want chop
- where is the nearest shop?
- where can i eat ?
- i need to eat
- are there eatries around ?
- where can i buy snacks ?
- I want to buy food
## intent:toilet_enquiry
- i neeed to pee
- where is the rest room?
- where is the toilet
- is there a nearby toilet?
- i they find toilet
- where is the bathroom
- i need to use the bathroom
- where can i pee
- i want peace
- i am pressed
"""

In [5]:
!touch nlu.md
%store nlu_md >nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [6]:
config = """
language: "en_core_web_md"
pipeline:
- name: "nlp_spacy"                   # loads the spacy language model
- name: "tokenizer_spacy"             # splits the sentence into tokens
- name: "ner_crf"                     # uses the pretrained spacy NER model
- name: "intent_featurizer_spacy"     # transform the sentence into a vector representation
- name: "intent_classifier_sklearn"   # uses the vector representation to classify using SVM
- name: "ner_synonyms"                # trains the synonyms
policies:
- name: "RulePolicy"
# a confidence >= core_fallback_threshold
core_fallback_threshold: 0.68                            # Confidence threshold for the `core_fallback_action_name` to apply.
core_fallback_action_name: "action_default_fallback"     # The action will apply if no other action was predicted with
enable_fallback_prediction: True                         # a confidence >= core_fallback_threshold
"""

In [9]:
!touch config.yml
%store config >config.yml

Writing 'config' (str) to file 'config.yml'.


In [4]:
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config


In [5]:
# loading the nlu training samples
training_data = load_data("nlu.md")
trainer = Trainer(config.load("config.yml"))

In [6]:
#training the nlu
interpreter = trainer.train(training_data)


Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_

In [7]:
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

In [8]:
import json
def pprint(o):
    print(json.dumps(o, indent=2))
pprint(interpreter.parse("where is hotel"))


{
  "intent": {
    "name": "accomodation_enquiry",
    "confidence": 0.7933114268106558
  },
  "entities": [],
  "intent_ranking": [
    {
      "name": "accomodation_enquiry",
      "confidence": 0.7933114268106558
    },
    {
      "name": "toilet_enquiry",
      "confidence": 0.07066984342886706
    },
    {
      "name": "restaurant_enquiry",
      "confidence": 0.041696646194211734
    },
    {
      "name": "goodbye",
      "confidence": 0.030620425055720074
    },
    {
      "name": "greet",
      "confidence": 0.017953616039899676
    },
    {
      "name": "directions",
      "confidence": 0.016016613529361554
    },
    {
      "name": "thanks",
      "confidence": 0.015675564431550403
    },
    {
      "name": "choosing_item",
      "confidence": 0.014055864509733996
    }
  ],
  "text": "where is hotel"
}


In [18]:
# Writing stories and saving it in the stories.md file
stories_md = """
##  Greeting path
* greet
- utter_greet
## directions path
* directions
- utter_menu
* choosing_destination
- utter_destination_received
## Enquires path
* accomodation_enquiry
- utter_enquiry
## thanks path
* thanks
- utter_thanks
## restaurant path
* restaurant_enquiry
- utter_restaurant_enquiry
## toilet path
* toilet_enquiry
- utter_toilet_location
## say goodbye
* goodbye
- utter_goodbye
## thanks
* thanks
- utter_thanks
## fallback rule
* bot_fallback
- action_default_fallback
"""
%store stories_md >stories.md


Writing 'stories_md' (str) to file 'stories.md'.


In [35]:
# Writing all the intents,slots,entities,actions and templates to domain.yml
domain_yml = """version: "3.0"
intents:
- greet
- goodbye
- directions
- choosing_item
- accomodation_enquiry
- restaurant_enquiry
- toilet_enquiry
- thanks
type: text
entities:
- group
actions:
- utter_greet
- utter_menu
- utter_destination_received
- utter_enquiry
- utter_toilet_location
- utter_restaurant_enquiry
- utter_goodbye
- utter_thanks
- utter_default
templates:
utter_greet:
- text: "Hey! Welcome to NRC Kubwa terminal . How may I help you ? "
utter_menu:
- text: "There is a bus terminal outside the station, it's just accross the road.  You can board a bus to the following destinations: \n\n 1. Wuse  2. Airport 3. Nyanya  4. Zuba  5. Asokoro  6. Maitama  7. Area one  8. Central area  8. Secretariat. \n\n Enter  the appropraite number to know when the next bus leaves the terminal. \n\n \n\n  You:"
utter_destination_received:
- text: " The next bus leaves in 15 minutes, do have a safe trip. "
utter_goodbye:
- text: " Thank you for choosing to transit with us. Waiting for your next visit. "
utter_enquiry:
- text: " Take a right turn from the exit, there are several hotels at your service. "
utter_toilet_location:
- text: " There is a toilet symbol on the extreme right from where you stand. "
utter_restaurant_enquiry:
- text: " There is a food court filled with a variety of rastaurants at the extreme left after the entrance. "
utter_thanks:
- text: " my pleasure "
utter_default:
- text: " I'm sorry, I didn't quite understand that. Could you rephrase? "
"""
%store domain_yml >domain.yml

Writing 'domain_yml' (str) to file 'domain.yml'.


In [9]:
from rasa_core.policies import FallbackPolicy, MemoizationPolicy,KerasPolicy

In [10]:
from rasa_core.agent import Agent


In [11]:
agent = Agent('domain.yml', policies=[MemoizationPolicy(), KerasPolicy(), ])

In [12]:
training_data = agent.load_data('stories.md')


Processed Story Blocks: 100%|██████████| 9/9 [00:00<00:00, 82.35it/s, # trackers=20]


In [13]:
agent.train(
training_data)
#validation_split=0.0,
#epochs=200)


Processed actions: 19it [00:00, 140.86it/s, # examples=19]


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, 5, 26)             0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                7552      
_________________________________________________________________
dense (Dense)                (None, 17)                561       
_________________________________________________________________
activation (Activation)      (None, 17)                0         
Total params: 8,113
Trainable params: 8,113
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Epoch 1/100
413/413 [================

In [14]:
agent.persist('models/dialogue')


In [47]:
!pip uninstall h5py
!pip install h5py==2.10.0


Found existing installation: h5py 3.1.0
Uninstalling h5py-3.1.0:
  Would remove:
    /usr/local/lib/python3.7/dist-packages/h5py-3.1.0.dist-info/*
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libaec-9c9e97eb.so.0.0.10
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libhdf5-00e8fae8.so.200.0.0
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libhdf5_hl-383c339f.so.200.0.0
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libsz-e7aa62f5.so.2.0.1
    /usr/local/lib/python3.7/dist-packages/h5py.libs/libz-eb09ad1d.so.1.2.3
    /usr/local/lib/python3.7/dist-packages/h5py/*
Proceed (y/n)? y
  Successfully uninstalled h5py-3.1.0
     |████████████████████████████████| 2.9 MB 4.4 MB/s 


In [15]:
from rasa_core.agent import Agent


In [16]:
agent = Agent.load('models/dialogue', interpreter=model_directory)



For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [17]:
print("Your bot is ready to talk! Type your messages here or send 'stop'" + "\n\n" + "You:")
while True:
    a = input()
    if a == 'stop':
        break
    responses = agent.handle_message(a)
    for response in responses:
        print(response["text"])


Your bot is ready to talk! Type your messages here or send 'stop'

You:
hi
Hey! Welcome to NRC Kubwa terminal . How may I help you ? 
I want to go pee
 There is a toilet symbol on the extreme right from where you stand. 
I want to eat something
 There is a food court filled with a variety of rastaurants at the extreme left after the entrance. 
I want to rest
There is a bus terminal outside the station, it's just accross the road.  You can board a bus to the following destinations: 1. Wuse  2. Airport 3. Nyanya  4. Zuba  5. Asokoro  6. Maitama  7. Area one  8. Central area  8. Secretariat. Enter  the appropraite number to know when the next bus leaves the terminal.
You:
1
 The next bus leaves in 15 minutes, do have a safe trip. 
2
 I'm sorry, I didn't quite understand that. Could you rephrase? 
3
 I'm sorry, I didn't quite understand that. Could you rephrase? 
terminal
There is a bus terminal outside the station, it's just accross the road.  You can board a bus to the following destinat